# Practica 2B (Ascensores)
##### Alberto García Doménech - Pablo Daurell Marina (Grupo 03)

In [1]:
# Restriccion: Una persona por ascensor
# El último ascensor siempre es rápido (se  mueve por plantas pares)
# estado_ini = ( (personas), (ascensores) )
estado_ini = (
    (2,4,2,8,1), (2,0,1)
)

# estado_goal = (personas)
estado_goal = (3,11,12,1,9)

# Acciones
# Llamar ascensor persona (mueve un ascensor vacio a una planta)
# Subir ascensor persona (sube una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar ascensor persona (baja una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)

In [2]:
from search import *

class Ascensores(Problem):
    
    def __init__(self, initial, goal):
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        personas = list(estado[0])
        ascensores = list(estado[1])
        accs = list()
        for i, persona in enumerate(personas):
            for j, ascensor in enumerate(ascensores):
                if j == len(ascensores)-1: # ASCENSOR RAPIDO
                    if persona != ascensor and persona%2 == 0:
                        accs.append(("LlamarRapido", i, j)) # Persona i (en planta par) llama a ascensor rapido
                    if persona == ascensor and int(persona) >0:
                        accs.append(("BajarRapido" , i, j)) # Bajar persona i en asccensor rapido (j)
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("SubirRapido", i, j)) # Subir persona i en ascensor rapido (j)
                else: # ASCENSOR NORMAL
                    if persona != ascensor:
                        accs.append(("Llamar", i, j))            # Persona i llama a asccensor j
                    if persona == ascensor and int(persona) > 0:
                        accs.append(("Bajar", i, j))             # Bajar persona i en ascensor j
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("Subir", i, j))             # Subir persona i en ascensor j
            
        return accs
    
    def result(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar" or accion[0] == "LlamarRapido":
            ascensores[a] = int(personas[p])
        elif accion[0] == "Bajar":
            ascensores[a] -= 1
            personas[p] -= 1
        elif accion[0] == "Subir":
            ascensores[a] += 1
            personas[p] += 1
        elif accion[0] == "BajarRapido":
            ascensores[a] -= 2
            personas[p] -= 2
        elif accion[0] == "SubirRapido":
            ascensores[a] += 2
            personas[p] += 2
        return (tuple(personas), tuple(ascensores))
    
    def goal_test(self, estado):
        return estado[0] == self.goal
    
    def coste_de_aplicar_accion(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar":
            coste = abs(personas[p] - ascensores[a])
        elif accion[0] == "Bajar":
            coste = 1
        elif accion[0] == "Subir":
            coste = 1
        elif accion[0] == "LlamarRapido":
            coste = abs(personas[p] - ascensores[a])/2
        elif accion[0] == "BajarRapido":
            coste = 1
        elif accion[0] == "SubirRapido":
            coste = 1
        return coste


In [3]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((2, 4, 2, 8, 1), (2, 0, 1))

In [4]:
p.goal

(3, 11, 12, 1, 9)

In [5]:
p.actions(p.initial)

[('Bajar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 0, 1),
 ('LlamarRapido', 0, 2),
 ('Llamar', 1, 0),
 ('Llamar', 1, 1),
 ('LlamarRapido', 1, 2),
 ('Bajar', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 2, 1),
 ('LlamarRapido', 2, 2),
 ('Llamar', 3, 0),
 ('Llamar', 3, 1),
 ('LlamarRapido', 3, 2),
 ('Llamar', 4, 0),
 ('Llamar', 4, 1),
 ('BajarRapido', 4, 2),
 ('SubirRapido', 4, 2)]

In [6]:
p.result(p.initial, ("Llamar", 0, 0))

((2, 4, 2, 8, 1), (2, 0, 1))

In [7]:
p.coste_de_aplicar_accion(p.initial, ("LlamarRapido", 1, 0))

1.0

### Heurísticas

In [8]:
# Numero de personas en una planta incorrecta a la de destino 
def descolocados(node):
    res = 0
    goal = estado_goal
    for i in range(len(node.state[0])):
        if goal[i] != node.state[0][i]:
            res += 1
    return res

# Distancia de cada persona a su planta destino
def distancias(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])
    return dist

# Distancia de cada persona a su planta destino (con ascensor rapido)
def distancias2(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])//2
        if goal[i] % 2 or state[i] % 2: #Si esta en una planta impar
            dist += 1;
    return dist

### Búsqueda heurística

In [9]:
from search import best_first_graph_search

##### *Dos personas, plantas pares*

In [50]:
estado_ini = ((0,0),(0,0))
estado_goal = (4,10)
p = Ascensores(estado_ini, estado_goal)

In [57]:
%%timeit
astar_search(p, descolocados).solution()

95.5 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%timeit
astar_search(p, distancias2).solution()

13.1 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%%timeit
best_first_graph_search(p, descolocados).solution()

9.36 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
%%timeit
best_first_graph_search(p, distancias2).solution()

511 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### *Dos personas, plantas impares*

In [15]:
estado_ini = ((1,3),(0,0))
estado_goal = (7,5)
p = Ascensores(estado_ini, estado_goal)

In [16]:
%%timeit
astar_search(p, descolocados).solution()

187 ms ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
astar_search(p, distancias2).solution()

162 ms ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
best_first_graph_search(p, descolocados).solution()

7.36 ms ± 932 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
best_first_graph_search(p, distancias2).solution()

3.16 ms ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### *Dos personas, plantas pares e impares*

In [20]:
estado_ini = ((2,4),(0,0))
estado_goal = (7,1)
p = Ascensores(estado_ini, estado_goal)

In [21]:
%%timeit
astar_search(p, descolocados).solution()

208 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
astar_search(p, distancias2).solution()

119 ms ± 5.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
best_first_graph_search(p, descolocados).solution()

4.52 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
best_first_graph_search(p, distancias2).solution()

3.9 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### *Tres personas*

In [29]:
estado_ini = ((2,4,1),(0,0))
estado_goal = (10,1,8)
p = Ascensores(estado_ini, estado_goal)

In [30]:
%%time
astar_search(p, descolocados).solution()

Wall time: 1min 56s


[('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('Bajar', 1, 0),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('LlamarRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1)]

In [31]:
%%time
astar_search(p, distancias2).solution()

Wall time: 5.03 s


[('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 2, 1),
 ('BajarRapido', 1, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('LlamarRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('Bajar', 1, 0)]

In [32]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 151 ms


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Subir', 1, 0),
 ('Bajar', 2, 0),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

In [33]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 5.99 ms


[('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 1, 0),
 ('LlamarRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 1, 1),
 ('Subir', 1, 0)]

##### *Dos personas, tres ascensores*

In [34]:
estado_ini = ((2,4),(0,1,6))
estado_goal = (10,1)
p = Ascensores(estado_ini, estado_goal)

In [36]:
%%time
astar_search(p, descolocados).solution()

Wall time: 13 s


[('LlamarRapido', 1, 2),
 ('BajarRapido', 1, 2),
 ('Llamar', 0, 1),
 ('Bajar', 1, 1),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2)]

In [35]:
%%time
astar_search(p, distancias2).solution()

Wall time: 716 ms


[('LlamarRapido', 1, 2),
 ('BajarRapido', 1, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('Llamar', 1, 1),
 ('Bajar', 1, 1)]

In [38]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 1.58 s


[('LlamarRapido', 0, 2),
 ('BajarRapido', 0, 2),
 ('Llamar', 0, 1),
 ('LlamarRapido', 1, 2),
 ('BajarRapido', 1, 2),
 ('BajarRapido', 1, 2),
 ('Subir', 1, 1),
 ('Llamar', 0, 1),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2)]

In [40]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 2.99 ms


[('LlamarRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('BajarRapido', 1, 2),
 ('BajarRapido', 1, 2),
 ('Llamar', 1, 1),
 ('LlamarRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('SubirRapido', 0, 2),
 ('LlamarRapido', 1, 2),
 ('Subir', 1, 1)]

##### *Cinco personas, dos ascensores*

In [41]:
estado_ini = ((2,4,1,8,1),(0,0))
estado_goal = (3,11,12,1,9)
p = Ascensores(estado_ini, estado_goal)

In [42]:
%%time
best_first_graph_search(p, distancias2).solution();

Wall time: 7.08 s


[('LlamarRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('LlamarRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('LlamarRapido', 3, 1),
 ('Llamar', 2, 0),
 ('Subir', 4, 0),
 ('Llamar', 3, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 4, 1),
 ('SubirRapido', 4, 1),
 ('SubirRapido', 4, 1),
 ('LlamarRapido', 3, 1),
 ('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 3, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 3, 1),
 ('Llamar', 1, 0),
 ('Subir', 1, 0),
 ('Llamar', 4, 0),
 ('Subir', 4, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 3, 0),
 ('Subir', 3, 0)]

In [43]:
%%time
best_first_graph_search(p, descolocados).solution();

Wall time: 2.41 s


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('Subir', 3, 0),
 ('Bajar', 2, 0),
 ('Llamar', 3, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Llamar', 0, 0),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 0, 1),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]